In [3]:
import sklearn
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
nltk.download('punkt')


train_set = r'C:\Users\Afnan\PycharmProjects\part 1\Dataset\train'
movie_train = load_files(train_set, shuffle=True)


test_set = r'C:\Users\Afnan\PycharmProjects\part 1\Dataset\test'
movie_test = load_files(test_set, shuffle=True)
with open(r'C:\Users\Afnan\PycharmProjects\part 1\Dataset\stop_words.txt') as file:
    stop_words_list = file.readlines()
    stop_words_list = [line.rstrip() for line in stop_words_list]



movie_vec = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize,stop_words=stop_words_list)
train_movie_counts = movie_vec.fit_transform(movie_train.data)

movie_vec = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize,stop_words=stop_words_list)
test_movie_counts = movie_vec.fit_transform(movie_test.data)



tfidf_transformer = TfidfTransformer()

train_movie_tfidf = tfidf_transformer.fit_transform(train_movie_counts)
test_movie_tfidf = tfidf_transformer.fit_transform(test_movie_counts)



docs_train, _, y_train, _ = train_test_split(train_movie_tfidf, movie_train.target, test_size = 0.01, random_state = 12)
docs_test, _, y_test, _= train_test_split(train_movie_tfidf, movie_train.target, test_size = 0.01, random_state = 12)

###################################################################################################
clf = MultinomialNB().fit(docs_train, y_train)
y_pred = clf.predict(docs_test)
print("Testing Accuracy is ",sklearn.metrics.accuracy_score(y_test, y_pred))


cm = confusion_matrix(y_test, y_pred)
print(cm)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Afnan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'m", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Testing Accuracy is  0.9122828282828283
[[11497   884]
 [ 1287 11082]]


In [13]:
import os
import math
import pandas as pd

'''According to the textbook,In most text classifica-
tion applications, however, using a stop word list doesn’t improve performance, and
so it is more common to make use of the entire vocabulary and not use a stop word
list.'''
pos_path = 'D:dataset\\train\\pos\\'
neg_path = 'D:dataset\\train\\neg\\'

test_neg = 'D:dataset\\test\\neg\\'
test_pos = 'D:dataset\\test\\pos\\'
f = open(r'C:\Users\Afnan\PycharmProjects\part 1\Dataset\stop_words.txt', "r", encoding="utf8")
stopWords = f.read().split()
f.close()

pos_listing = os.listdir(pos_path)
neg_listing = os.listdir(neg_path)

test_listing_n = os.listdir(test_neg)
test_listing_p = os.listdir(test_pos)

vocab = set()

pos_totalWords = []
neg_totalWords = []

# positive files
for file in pos_listing:

    f = open(pos_path + file, "r", encoding="utf8")

    allWords = f.read().split()

    for word in allWords:
        vocab.add(word.lower())
        pos_totalWords.append(word.lower())

    f.close()

# negative files
for file in neg_listing:

    f = open(neg_path + file, "r", encoding="utf8")

    allWords = f.read().split()

    for word in allWords:
        vocab.add(word.lower())
        neg_totalWords.append(word.lower())

    f.close()

for word in stopWords:
    try:
        pos_totalWords.remove(word)
        neg_totalWords.remove(word)
        vocab.remove(word)
    except:
        pass

pos_dict = dict.fromkeys(vocab, 0)
neg_dict = dict.fromkeys(vocab, 0)

for p in pos_totalWords:
    key = p
    if key in vocab:
        pos_dict[p] += 1

for p in neg_totalWords:
    key = p
    if key in vocab:
        neg_dict[p] += 1

print("done frequency counting")

pos_prob = dict.fromkeys(vocab, 0)
neg_prob = dict.fromkeys(vocab, 0)

for word in pos_dict:
    pos_prob[word] = (pos_dict[word] + 1) / (len(pos_totalWords) + len(vocab))

print("done positive probability dictionary")

for word in neg_dict:
    neg_prob[word] = (neg_dict[word] + 1) / (len(neg_totalWords) + len(vocab))

print("negative dictionary")

pos_probability = 0
neg_probability = 0
pos_review_pos_test = 0
neg_review_pos_test = 0
pos_review_neg_test = 0
neg_review_neg_test = 0

for file in test_listing_p:
    f = open(test_pos + file, "r", encoding="utf8")

    allWords = f.read().split()

    for word in allWords:
        try:
            pos_probability += math.log(pos_prob[word])  # prior probability is constant so it doesnt matter anyway
            neg_probability += math.log(neg_prob[word])
        except KeyError:
            pos_probability += 0
            neg_probability += 0

    if pos_probability > neg_probability:
        pos_review_pos_test += 1
    else:
        neg_review_pos_test += 1

    pos_probability = 0
    neg_probability = 0

for file in test_listing_n:
    f = open(test_neg + file, "r", encoding="utf8")

    allWords = f.read().split()

    for word in allWords:
        try:
            pos_probability += math.log(pos_prob[word])
            neg_probability += math.log(neg_prob[word])
        except KeyError:
            pos_probability += 0
            neg_probability += 0

    if pos_probability > neg_probability:
        pos_review_neg_test += 1
    else:
        neg_review_neg_test += 1

    pos_probability = 0
    neg_probability = 0

print("")
print("Results")
print("=======")
cfm = pd.DataFrame([[pos_review_pos_test, pos_review_neg_test], [neg_review_pos_test, neg_review_neg_test]],
                   columns=('Gold positive', 'Gold negative'), index=('Predicted positive', 'Predicted negative'))
print(cfm)
ac = (pos_review_pos_test + neg_review_neg_test) / (
            pos_review_pos_test + pos_review_neg_test + neg_review_neg_test + neg_review_pos_test)
print("accuracy={}".format(ac))


done frequency counting
done positive probability dictionary
negative dictionary

Results
                    Gold positive  Gold negative
Predicted positive           9802           1220
Predicted negative           2698          11280
accuracy=0.84328
